In [11]:
import pandas as pd
import numpy as np

In [12]:
predict_results={'file1':'/kaggle/input/dev-set-predictions/banglabert.tsv', 'file2':'/kaggle/input/dev-set-predictions/banglishbert.tsv', 'file3':'/kaggle/input/dev-set-predictions/hatespeech.tsv' }
accuracy={'file1':0.7125,'file2':0.7021, 'file3':0.696492135524749}
df=pd.read_csv(predict_results['file1'], delimiter='\t')
df.rename(columns={'label': 'labels_model1'}, inplace=True)
for i in range(2, 4):
    current_file='file'+str(i)
    temp_df=pd.read_csv(predict_results[current_file], delimiter='\t')
    df['labels_model'+str(i)]=temp_df['label'].copy()
df.head

<bound method NDFrame.head of          id labels_model1 labels_model2 labels_model3
0     30670      Negative      Negative      Negative
1      4125      Negative      Negative      Negative
2     27077      Positive      Positive      Negative
3     17552      Positive      Positive      Negative
4      4137      Negative      Negative      Positive
...     ...           ...           ...           ...
3421  22376      Negative      Negative      Negative
3422   1091       Neutral      Negative      Negative
3423  17792      Positive      Positive      Positive
3424  23487      Negative      Negative      Negative
3425  10751       Neutral       Neutral       Neutral

[3426 rows x 4 columns]>

In [13]:
labeltonum={0:'Negative', 1:'Positive', 2:'Neutral'}
def majority_vote(df):
    labels=df[1:4]
    counts=[0,0,0]
    for x in labels:
        if x=='Negative':
            counts[0]+=1
        if x=='Positive':
            counts[1]+=1
        if x=='Neutral':
            counts[2]+=1
    final_label=labeltonum[np.argmax(counts)]
    return final_label
df['majority vote']=df.apply(majority_vote, axis=1)
df.head

<bound method NDFrame.head of          id labels_model1 labels_model2 labels_model3 majority vote
0     30670      Negative      Negative      Negative      Negative
1      4125      Negative      Negative      Negative      Negative
2     27077      Positive      Positive      Negative      Positive
3     17552      Positive      Positive      Negative      Positive
4      4137      Negative      Negative      Positive      Negative
...     ...           ...           ...           ...           ...
3421  22376      Negative      Negative      Negative      Negative
3422   1091       Neutral      Negative      Negative      Negative
3423  17792      Positive      Positive      Positive      Positive
3424  23487      Negative      Negative      Negative      Negative
3425  10751       Neutral       Neutral       Neutral       Neutral

[3426 rows x 5 columns]>

In [14]:
def weighted_vote(df):
    labels=df[1:4]
    counts=[0,0,0]
    for i in range(3):
        if labels[i]=='Negative':
            counts[0]+=accuracy['file'+str(i+1)]
        if labels[i]=='Positive':
            counts[1]+=accuracy['file'+str(i+1)]
        if labels[i]=='Neutral':
            counts[2]+=accuracy['file'+str(i+1)]
    final_label=labeltonum[np.argmax(counts)]
    return final_label
df['weighted vote']=df.apply(weighted_vote, axis=1)
df.head

<bound method NDFrame.head of          id labels_model1 labels_model2 labels_model3 majority vote  \
0     30670      Negative      Negative      Negative      Negative   
1      4125      Negative      Negative      Negative      Negative   
2     27077      Positive      Positive      Negative      Positive   
3     17552      Positive      Positive      Negative      Positive   
4      4137      Negative      Negative      Positive      Negative   
...     ...           ...           ...           ...           ...   
3421  22376      Negative      Negative      Negative      Negative   
3422   1091       Neutral      Negative      Negative      Negative   
3423  17792      Positive      Positive      Positive      Positive   
3424  23487      Negative      Negative      Negative      Negative   
3425  10751       Neutral       Neutral       Neutral       Neutral   

     weighted vote  
0         Negative  
1         Negative  
2         Positive  
3         Positive  
4         Ne

In [16]:
majority_vote_df=df[['id','majority vote']]
majority_vote_df.rename(columns={'majority vote': 'label'}, inplace=True)
majority_vote_df.to_csv('majority_prediction.tsv', sep='\t')
weighted_vote_df=df[['id','weighted vote']]
weighted_vote_df.rename(columns={'weighted vote': 'label'}, inplace=True)
weighted_vote_df.to_csv('weighted_prediction.tsv', sep='\t')

/tmp/ipykernel_32/1845264061.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  majority_vote_df.rename(columns={'majority vote': 'label'}, inplace=True)
/tmp/ipykernel_32/1845264061.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weighted_vote_df.rename(columns={'weighted vote': 'label'}, inplace=True)
